<a href="https://colab.research.google.com/github/sdshaheer/Major-project1/blob/main/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch 
from torchvision import transforms , models 
from PIL import Image 
import matplotlib.pyplot as plt
import numpy as np

device = ("cuda" if torch.cuda.is_available() else "cpu")


model = models.vgg19(pretrained=True).features
for p in model.parameters():
    p.requires_grad = False
model.to(device)
model

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [ ]:
transform = transforms.Compose([transforms.Resize(300),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [ ]:
def model_activations(input,model):
    layers = {
    '0' : 'conv1_1',
    '5' : 'conv2_1',
    '10': 'conv3_1',
    '19': 'conv4_1',
    '21': 'conv4_2',
    '28': 'conv5_1'
    }
    features = {}
    x = input
    x = x.unsqueeze(0)
    for name,layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x 

    print(features)
    return features

In [ ]:
def imcnvt(image):
    x = image.to("cpu").clone().detach().numpy().squeeze()
    x = x.transpose(1,2,0)
    x = x*np.array((0.5,0.5,0.5)) + np.array((0.5,0.5,0.5))
    return np.clip(x,0,1)

In [ ]:
def gram_matrix(imgfeature):
    _,d,h,w = imgfeature.size()
    imgfeature = imgfeature.view(d,h*w)
    gram_mat = torch.mm(imgfeature,imgfeature.t())
    
    return gram_mat

In [ ]:
def load(image_path,x,y):
    image = Image.open(image_path)
    image = image.resize((y,x))
    return image

In [ ]:
def getname(filename):
  return filename.split(".")[0]

### Storage for images

In [ ]:
if not os.path.exists("/content/static"):
  os.makedirs("/content/static")

if not os.path.exists("/content/static/uploads"):
  os.makedirs("/content/static/uploads")

if not os.path.exists("/content/static/uploads/video_files"):
  os.makedirs("/content/static/uploads/video_files")
if not os.path.exists("/content/static/uploads/photo_files"):
  os.makedirs("/content/static/uploads/photo_files")

if not os.path.exists("/content/static/uploads/photo_files/input"):
  os.makedirs("/content/static/uploads/photo_files/input")
if not os.path.exists("/content/static/uploads/photo_files/output"):
  os.makedirs("/content/static/uploads/photo_files/output")

if not os.path.exists("/content/static/uploads/video_files/input"):
  os.makedirs("/content/static/uploads/video_files/input")
if not os.path.exists("/content/static/uploads/video_files/output"):
  os.makedirs("/content/static/uploads/video_files/output")


In [ ]:
style_wt_meas = {"conv1_1" : 1.0, 
                 "conv2_1" : 0.8,
                 "conv3_1" : 0.4,
                 "conv4_1" : 0.2,
                 "conv5_1" : 0.1}

content_wt = 100
style_wt = 1e8

epochs = 400

# Photo Styling

In [ ]:
def train_photostyle(video_or_photo,content_filename,style_filename,target,content_features,style_grams,optimizer,epochs):
  for i in range(1,epochs+1):
    target_features = model_activations(target,model)
    content_loss = torch.mean((content_features['conv4_2']-target_features['conv4_2'])**2)

    style_loss = 0
    for layer in style_wt_meas:
        style_gram = style_grams[layer]
        target_gram = target_features[layer]
        _,d,w,h = target_gram.shape
        target_gram = gram_matrix(target_gram)

        style_loss += (style_wt_meas[layer]*torch.mean((target_gram-style_gram)**2))/d*w*h
    
    total_loss = content_wt*content_loss + style_wt*style_loss 
    
    #if i%10==0:       
        #print("epoch ",i," ", total_loss)

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
  if video_or_photo=="photo":
    combined_name=content_filename+style_filename
    path="/content/static/uploads/photo_files/output/"+combined_name
    plt.imsave(path+'.png',imcnvt(target),format='png')
    print("training completed")
    return path,combined_name
  else:
    path="/content/styled_frames/"+content_filename
    plt.imsave(path+'.png',imcnvt(target),format='png')


In [ ]:
def style_photo(content_filepath,style_filepath,content_filename,style_filename):
  content = Image.open(content_filepath).convert("RGB")
  content = transform(content).to(device)
  lis=content.shape

  style = load(style_filepath,lis[1],lis[2])
  style = transform(style).to(device)

  target = content.clone().requires_grad_(True).to(device)

  style_features = model_activations(style,model)
  content_features = model_activations(content,model)


  style_grams = {layer:gram_matrix(style_features[layer]) for layer in style_features}

  optimizer = torch.optim.Adam([target],lr=0.07)

  path,combined_name=train_photostyle("photo",content_filename,style_filename,target,content_features,style_grams,optimizer,epochs)
  return path,combined_name




## Video styling

In [ ]:
def divide_frames(path):
  cam = cv2.VideoCapture(path)
  
  try:
      # creating a folder named data
      if not os.path.exists('video_frames'):
          os.makedirs('video_frames')
  except OSError:
      print ('Error: Creating directory of data')
    
  # frame
  currentframe = 0
    
  while(True):
        
      # reading from frame
      ret,frame = cam.read()
    
      if ret:
          # if video is still left continue creating images
          name = './video_frames/frame' + str(currentframe) + '.jpg'
          print ('Creating...' + name)

          # writing the extracted images
          cv2.imwrite(name, frame)
    
          # increasing counter so that it will
          # show how many frames are created
          currentframe += 1
      else:
          break
    
  # Release all space and windows once done
  cam.release()
  cv2.destroyAllWindows()

  return "/content/video_frames"

In [ ]:
def convert_to_video(video_filename,style_filename):
  path="/content/styled_frames"
  out_path="/content/static/uploads/video_files/output/"
  video_name=video_filename+style_filename+".mp4"
  full_path=out_path+video_name

  img=[]
  num_of_images = len(os.listdir(path))
  for i in range(num_of_images):
    curr=path+"/frame"+str(i)+".png"
    img.append(curr)

  cv2_fourcc=cv2.VideoWriter_fourcc(*'mp4v')
  frame=cv2.imread(img[0])
  size=list(frame.shape)
  del size[2]
  size.reverse()
  
  video=cv2.VideoWriter(full_path,cv2_fourcc,15,size)

  for i in range(len(img)):
    video.write(cv2.imread(img[i]))
  video.release()

  return video_name

In [ ]:
def style_video(video_filename,frames,style_filepath,style_filename):
  image1=frames+"/frame0.jpg"
  content = Image.open(image1).convert("RGB")
  content = transform(content).to(device)
  lis=content.shape

  style = load(style_filepath,lis[1],lis[2])
  style = transform(style).to(device)

  style_features = model_activations(style,model)
  style_grams = {layer:gram_matrix(style_features[layer]) for layer in style_features}

  # sorting frames
  path=frames+"/"
  images=[]
  num_of_images = len(os.listdir(path))
  for i in range(num_of_images):
    curr=path+"frame"+str(i)+".jpg"
    images.append(curr)

  if not os.path.exists("/content/styled_frames"):
    os.makedirs("/content/styled_frames")

  for j in range(len(images)):
    path=images[j]
    lis_path=path.split("/")
    content = Image.open(path).convert("RGB")
    content = transform(content).to(device)
    target = content.clone().requires_grad_(True).to(device)
    content_features = model_activations(content,model)
    optimizer = torch.optim.Adam([target],lr=0.07)
    train_photostyle("video",getname(lis_path[-1]),style_filename,target,content_features,style_grams,optimizer,epochs)
  return convert_to_video(video_filename,style_filename)

# Web Interface

In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=dd16f365db98b8d10a0b1b9dd45dcbb4e0d1a66aaa1ae30806366d758812aceb
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask,render_template
from pyngrok import ngrok
from flask import flash, request, redirect, url_for
import urllib.request
import os
from werkzeug.utils import secure_filename

In [ ]:
%mkdir templates -p

### Web page for uploading photo and style images

In [ ]:
%%writefile templates/photo.html

<!DOCTYPE html>
<html lang="en">
<head>
<title>Python Flask Upload and display image</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" /> 
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
</head>
<body>
<p><h1 align="center">NEURAL STYLE TRANSFER</h1></p>
<div class="container">
<div class="row">
    <h2>Select a file to upload</h2>
    
    <form method="post" action="/upload_image" enctype="multipart/form-data">
        <div>
              <label for="files">select content image</label>
              <input type="file" name="content_file" class="form-control" autocomplete="off" required>
        </div>
        <br>
        <div>
              <label for="files">select style image</label>
              <input type="file" name="style_file" class="form-control" autocomplete="off" required>
        </div>
        <br>
        <div>
            <input type="submit" value="Submit" class="btn btn-info">
        </div>
    </form>
</div>
</div>
</body>
</html>

Writing templates/photo.html


### Web page for uploading video and style images

In [ ]:
%%writefile templates/video.html
<!DOCTYPE html>
<html lang="en">
<head>

<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" /> 
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
</head>
<body>
<p><h1 align="center">Neural Style Transfer</h1></p>
<div class="container">
<div class="row">
    <h2>Select a file to upload</h2>
    
    <form method="post" action="/upload_video" enctype="multipart/form-data">
        <div>
              <label for="files">select video file</label>
              <input type="file" name="video_file" class="form-control" autocomplete="off" required>
        </div>
        <br>
        <div>
              <label for="files">select style image</label>
              <input type="file" name="style_file" class="form-control" autocomplete="off" required>
        </div>
        <br>
        <div>
            <input type="submit" value="Submit" class="btn btn-info">
        </div>
    </form>
</div>
</div>
</body>
</html>

Writing templates/video.html


### Home page

In [ ]:
%%writefile templates/home.html

<!DOCTYPE html>
<html lang="en">
<head>
<style>
* {
  box-sizing: border-box;
}

.button {
  background-color: #008CBA; /* Green */
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  border-radius:25px;
}

.b1{
  display:inline-block;
  padding-left:25%;
}

</style>
</head>
<body>
<p><h1 align="center">Neural Style Transfer</h1></p>

<p style="color:black;font-size:20px;margin:2%">

Neural style transfer (NST) refers to a class of software algorithms that manipulate digital images, or videos, in order to adopt the appearance or visual style of another image.
NST algorithms are characterized by their use of deep neural networks for the sake of image transformation.
Common uses for NST are the creation of artificial artwork from photographs, for example by transferring the appearance of famous paintings to user-supplied photographs.
Several notable mobile apps use NST techniques for this purpose,
including DeepArt and Prisma. This method has been used by artists and designers around the globe to develop new artwork based on existent style(s).

</p>

<p style="color:black;font-size:20px;margin:2%">
Neural style transfer is an optimization technique used to take three images, a content image, a style reference image (such as an artwork by a famous painter), 
and the input image you want to style — and blend them together such that the input image is transformed to look like the content image, but “painted” in the style of the style image.
</p>



<div class="b1">
      <form method="post" action="/transfer_to_photopage" enctype="multipart/form-data">
              <div>
                  <input type="submit" value="Style Photo" class="button">
              </div>
      </form>
</div>
<div class="b1">
      <form method="post" action="/transfer_to_videopage" enctype="multipart/form-data">
              <div>
                  <input type="submit" value="Style Video" class="button">
              </div>
      </form>
</div>

</body>
</html>



Writing templates/home.html


### Web page for displaying styled video

In [ ]:
%%writefile templates/styled_video.html
<!DOCTYPE html>
<html lang="en">
<head>
<title>Python Flask Upload and display image</title>
<style>
* {
  box-sizing: border-box;
}

.column {
  float: left;
  width: 33.33%;
  padding: 5px;
}

/* Clearfix (clear floats) */
.row::after {
  content: "";
  clear: both;
  display: table;
}

/* Responsive layout - makes the three columns stack on top of each other instead of next to each other */
@media screen and (max-width: 500px) {
  .column {
    width: 100%;
  }
}
</style>

</head>
<body>
<p><h1 align="center">Neural Style Transfer</h1></p>
<div class="row">

      {% if style %} 
        <div class="column">
          <img src="{{url_for('static',filename ="uploads/video_files/input"+style)}}" alt="style" height="240px" width="300px">
        </div>
      {% endif %}

      {% if video %} 
        <div class="column">
          <video width="320" height="240" controls>
              <source src="{{url_for('static',filename ="uploads/video_files/input"+video)}}" type="video/mp4">
              Your browser does not support the video tag.
          </video>
        </div>
      {% endif %}
</div>
</body>
</html>

Writing templates/styled_video.html


### Web page for displaying styled photo

In [ ]:
%%writefile templates/styled_photo.html

<!DOCTYPE html>
<html lang="en">
<head>
<style>
* {
  box-sizing: border-box;
}

.column {
  float: left;
  width: 33.33%;
  padding: 5px;
}

/* Clearfix (clear floats) */
.row::after {
  content: "";
  clear: both;
  display: table;
}

/* Responsive layout - makes the three columns stack on top of each other instead of next to each other */
@media screen and (max-width: 500px) {
  .column {
    width: 100%;
  }
}
</style>

</head>
<body>
<p><h1 align="center">Neural Style Transfer</h1></p>
<div class="row">
      {% if content %} 
        <div class="column">
          <img src="{{url_for('static',filename ="uploads/photo_files/input/"+content)}}" alt="content image" height="240px" width="300px">
        </div>
      {% endif %}

      {% if style %} 
        <div class="column">
          <img src="{{url_for('static',filename ="uploads/photo_files/input/"+style)}}" alt="style" height="240px" width="300px">
        </div>
      {% endif %}

      {% if combined %} 
        <div class="column">
          <img src="{{url_for('static',filename ="uploads/photo_files/output/"+combined)}}" alt="combined image" height="240px" width="300px">
        </div>
      {% endif %}
</div>
</body>
</html>

Writing templates/styled_photo.html


In [ ]:

from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

ngrok.set_auth_token("2H5LzxpO65jU4Nr0V2HrK1Y0kTw_2C5DWLx75FESdwFhg88Qf")
#public_url=ngrok.connect(port_no).public_url
app = Flask(__name__, template_folder='/content/templates', static_folder='/content/static')
run_with_ngrok(app)

UPLOAD_FOLDER = "static/uploads/"
 
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])
ALLOWED_VIDEO_EXTENSIONS = set(['mp4','mpg','mpv'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def allowed_video_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_VIDEO_EXTENSIONS

@app.route('/')
def main():
    return render_template('home.html')

@app.route('/transfer_to_photopage',methods=['GET','POST'])
def transfer_to_photopage():
  return render_template('photo.html')

@app.route('/transfer_to_videopage',methods=['GET','POST'])
def transfer_to_videopage():
  return render_template('video.html')

@app.route('/upload_video',methods=['GET','POST'])
def upload_video():
    if 'video_file' not in request.files:
        flash('No video file part')
        return redirect(request.url)
    if 'style_file' not in request.files:
        flash('No style file part')
        return redirect(request.url)

    #################
    video_file = request.files['video_file']
    style_file = request.files['style_file']

    #################
    if video_file.filename == '':
        flash('No video selected for uploading')
        return redirect(request.url)
    if style_file.filename == '':
        flash('No style image selected for uploading')
        return redirect(request.url)

    ################
    if video_file and style_file and allowed_video_file(video_file.filename) and allowed_file(style_file.filename):
        video_filename = secure_filename(video_file.filename)
        video_filepath=os.path.join(app.config['UPLOAD_FOLDER'],"video_files/input/")
        video_filepath=os.path.join(video_filepath, video_filename)
        video_file.save(video_filepath)
        print('upload_image filename: ' + video_filename)

        #flash('Image successfully uploaded and displayed below')
        style_filename = secure_filename(style_file.filename)
        style_filepath=os.path.join(app.config['UPLOAD_FOLDER'],"video_files/input/")
        style_filepath=os.path.join(style_filepath,style_filename)
        style_file.save(style_filepath)

        print("dividing frames")
        frames=divide_frames(video_filepath)
        print("completed")
        video_name=style_video(video_filename,frames,style_filepath,style_filename)
    
        return render_template('styled_video.html',video=video_name,style=style_filename)



@app.route('/upload_image', methods=['GET','POST'])
def upload_image():
    if 'content_file' not in request.files:
        flash('No content file part')
        return redirect(request.url)
    if 'style_file' not in request.files:
        flash('No style file part')
        return redirect(request.url)
    #################
    content_file = request.files['content_file']
    style_file = request.files['style_file']

    #################
    if content_file.filename == '':
        flash('No content image selected for uploading')
        return redirect(request.url)
    if style_file.filename == '':
        flash('No style image selected for uploading')
        return redirect(request.url)
    
    #################
    if content_file and style_file and allowed_file(content_file.filename) and allowed_file(style_file.filename):
        content_filename = secure_filename(content_file.filename)
        content_filepath=os.path.join(app.config['UPLOAD_FOLDER'],"photo_files/input/" )
        content_filepath=os.path.join(content_filepath, content_filename)
        content_file.save(content_filepath)
        print('upload_image filename: ' + content_filename)

        #flash('Image successfully uploaded and displayed below')
        style_filename = secure_filename(style_file.filename)
        style_filepath=os.path.join(app.config['UPLOAD_FOLDER'],"photo_files/input/")
        style_filepath=os.path.join(style_filepath, style_filename)
        style_file.save(style_filepath)
        final_filenames=[content_filename,style_filename]
        print('upload_image filename: ' + style_filename)

        passed_contentname=getname(content_filename)
        passed_stylename=getname(style_filename)
        
        combined_filepath,combined_filename=style_photo(content_filepath,style_filepath,passed_contentname,passed_stylename)
        return render_template('styled_photo.html',content=content_filename,style=style_filename,combined=combined_filename+".png")
    else:
        flash('Allowed image types are - png, jpg, jpeg, gif')
        return redirect(request.url)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1ef6-34-67-99-186.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [05/Nov/2022 03:51:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2022 03:51:59] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2022 03:52:09] "POST /transfer_to_photopage HTTP/1.1" 200 -


Streaming output truncated to the last 5000 lines.
           0.0000e+00, 2.2334e-01],
          [0.0000e+00, 0.0000e+00, 1.0189e-01,  ..., 1.4948e-01,
           0.0000e+00, 3.4735e-01],
          [4.2793e-01, 5.9328e-01, 7.7102e-01,  ..., 6.1100e-01,
           4.7172e-01, 4.1438e-01],
          ...,
          [6.6722e-01, 2.5482e-01, 2.2616e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.0314e-01, 1.9698e-01, 9.3489e-01,  ..., 0.0000e+00,
           0.0000e+00, 2.2805e-01],
          [4.4091e-01, 8.1163e-01, 8.3831e-01,  ..., 1.0298e+00,
           6.6661e-01, 5.3773e-01]]]], device='cuda:0',
       grad_fn=<ReluBackward0>), 'conv2_1': tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.9047, 0.0000, 0.0000,  ..., 4.5722, 0.4080, 0.0000],
          ...,
          [2.9349, 2.6208, 1.1954,  ..., 2.4206, 4.1851, 2.0979],
          [1.6410, 0.0000, 1.0402,  ..., 2.9580, 2

INFO:werkzeug:127.0.0.1 - - [05/Nov/2022 03:54:26] "POST /upload_image HTTP/1.1" 200 -


training completed


INFO:werkzeug:127.0.0.1 - - [05/Nov/2022 03:54:27] "GET /static/uploads/photo_files/input/content1.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2022 03:54:29] "GET /static/uploads/photo_files/input/style6.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2022 03:54:29] "GET /static/uploads/photo_files/output/content1style6.png HTTP/1.1" 200 -
